In [2]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))


from src.config.setup import SetupConfig
from src.data_workflow.create_tables.create_tables import retrieve_all_table_data

import pandas as pd

In [3]:
# Read View
view_di = retrieve_all_table_data('view_data_integration')

In [4]:
# Shape
view_di.shape

(1016193, 30)